In [ ]:
# Align force data
reaction_yforce_data_aligned = np.zeros_like(reaction_yforce_data)
for i in range(num_subjects):
    for j in range(num_blocks):
        for k in range(num_trials):
            if np.isnan(agent_reaction_decision_time[i,j,k]):
                break
            x = int(agent_reaction_decision_time[i,j,k])
            reaction_yforce_data_aligned[i,j,k,:] = np.roll(reaction_yforce_data[i,j,k,:],-x)
reaction_yforce_data_aligned[...,1000:] = np.nan

In [ ]:
#* Split into react and gamble trials for first block
shutoff_mask = trial_type_array==0
go_mask = trial_type_array==1
force_react_mask      = np.repeat(go_mask[...,np.newaxis],5000,axis=3)
force_gamble_mask     = np.repeat(shutoff_mask[...,np.newaxis],5000,axis=3)

react_yforce_data = reaction_yforce_data_aligned[force_react_mask].reshape(num_subjects,2,50,5000)   
gamble_yforce_data = reaction_yforce_data_aligned[force_gamble_mask].reshape(num_subjects,2,50,5000)   

react_yforce_data_mixed = react_yforce_data[:,0,...]
react_yforce_data_only_react = react_yforce_data[:,1,...]

gamble_yforce_data_mixed = gamble_yforce_data[:,0,...]
gamble_yforce_data_only_gamble = gamble_yforce_data[:,1,...]

In [ ]:
class Data():
    def __init__(self,data):
        self.data = data
        self.num_subjects = 7
        self.get_max_force_percentiles()
    def get_max_force_percentiles(self):
        #* Get average across all trials for each subject, each block
        self.mean_force = np.nanmean(self.data,axis=1)

        #* Find max peak in average signal and find 0.25 and 0.75 of the averaged force values
        self.max_mean_force = np.nanmax(self.mean_force,axis=1)
        self.index_max_mean_force = np.nanargmax(self.mean_force,axis=1)
        self.max25 = 0.25*self.max_mean_force
        self.max75 = 0.75*self.max_mean_force
        self.max25_timepoint = np.zeros((num_subjects))
        self.max75_timepoint = np.zeros_like(self.max25_timepoint)
        for i in range(self.num_subjects):
            a,_ = min(enumerate(self.mean_force[i,:self.index_max_mean_force[i]]), key=lambda x: abs(x[1]-self.max25[i])) # Enumerate mean force to get timepoint and value, only slice up to the max value, then find where it's closest to 25percent of max value
            self.max25_timepoint[i] = a
            b,_ = min(enumerate(self.mean_force[i,:self.index_max_mean_force[i]]), key=lambda x: abs(x[1]-self.max75[i]))
            self.max75_timepoint[i] = b
        self.x1vals = self.max25_timepoint
        self.x2vals = self.max75_timepoint
        self.y1vals = self.max25
        self.y2vals = self.max75
        self.slopes = (self.y2vals - self.y1vals)/(self.x2vals - self.x1vals)
        self.intercepts = self.y2vals - self.slopes*self.x2vals
        self.time_at_zero = -self.intercepts/self.slopes
        self.player_reaction_time_force = self.time_at_zero

In [ ]:
def plot(objects,label):
    colors = cm.rainbow(np.linspace(0, 1, 4))
    
    
    for i in range(num_subjects):
        fig,ax = plt.subplots(dpi=170)
        for j,o in enumerate(objects):
            x1 = o.mean_force[i,:]
            x1line = o.slopes[i]*np.arange(0,1000) + o.intercepts[i]

            ax.plot(np.arange(0,1000),x1[:1000],c =colors[j],label=label[j])
            ax.scatter(o.max25_timepoint[i],o.max25[i],c='grey')
            ax.scatter(o.max75_timepoint[i],o.max75[i],c='grey')
            ax.plot(x1line,c=colors[j],ls='--')

            ax.scatter(o.index_max_mean_force[i],o.max_mean_force[i],marker='x')

            ax.set_ylim(0,max(o.max_mean_force+1))
            # ax.set_xlim(0,800)
            ax.legend()
        plt.show()

In [ ]:
react_mixed = Data(react_yforce_data_mixed)
react_only_react = Data(react_yforce_data_only_react)
gamble_mixed = Data(gamble_yforce_data_mixed)
gamble_only_gamble = Data(gamble_yforce_data_only_gamble)
# mean_react_mixed, max_mean_react_mixed, index_max_mean_react_mixed, max25_react_mixed, max75_react_mixed, max25_timepoint_react_mixed, max75_timepoint_react_mixed = get_max_force_percentiles(react_yforce_data_mixed)
# mean_only_react, max_mean_only_react, index_max_mean_only_react, max25_only_react, max75_only_react, max25_timepoint_only_react, max75_timepoint_only_react = get_max_force_percentiles(react_yforce_data_only_react)
# mean_only_gamble, max_mean_only_gamble, index_max_mean_only_gamble, max25_only_gamble, max75_only_gamble, max25_timepoint_only_gamble, max75_timepoint_only_gamble = get_max_force_percentiles(gamble_yforce_data_only_gamble)
# mean_gamble_mixed, max_mean_gamble_mixed, index_max_mean_gamble_mixed, max25_gamble_mixed, max75_gamble_mixed, max25_timepoint_gamble_mixed, max75_timepoint_gamble_mixed = get_max_force_percentiles(gamble_yforce_data_mixed)
o = [react_mixed,react_only_react,gamble_mixed,gamble_only_gamble]